In [1]:
!pip install unrar
!unrar x /content/train.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/train.rar

Extracting  train.csv                                                     14% 29% 44% 59% 74% 89%100%  OK 
All OK


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/content/train.csv')
print(data)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
data = data.dropna()

In [7]:
X = data.drop('label', axis=1)

In [8]:
y = data['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [10]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.15.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
voc_size = 5000

# One Hot Representation

In [15]:
masseges = X.copy()

In [16]:
masseges['title'][7]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [17]:
masseges

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(masseges)):
    review = re.sub('[^a-zA-Z]', ' ', masseges['title'].iloc[i])
    review = review.lower()
    review = review.split()
    a=[]
    for word in review:
      if word not in stopwords.words('english'):
        a.append(PorterStemmer().stem(word))
    review = ' '.join(a)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr = []
for words in corpus:
  onehot_repr.append(one_hot(words,voc_size))
print(onehot_repr)

[[1147, 2444, 2191, 1587, 3249, 4732, 4018, 3799, 2045, 4992], [453, 2785, 3690, 4958, 3997, 3862, 4382], [2595, 4439, 3051, 3532], [1645, 955, 152, 2698, 1514, 4257], [354, 3997, 3381, 2400, 2971, 3572, 3997, 4577, 2093, 127], [3606, 4572, 2689, 2705, 2794, 4723, 1785, 2591, 1658, 3213, 444, 1815, 2487, 1750, 4382], [774, 183, 1473, 4230, 2708, 1777, 2233, 3810, 4290, 4612, 1174], [500, 2459, 1586, 209, 3226, 4856, 4723, 312, 4290, 4612, 1174], [1474, 1819, 3292, 4246, 252, 638, 2522, 4902, 4723, 2345], [4941, 2757, 338, 2968, 3326, 2666, 2903, 4654], [1390, 93, 891, 961, 4844, 1444, 4928, 4099, 4082, 2494, 4995], [2698, 1841, 3249, 638, 4723, 3226], [3467, 214, 3732, 304, 2079, 2385, 1378, 4811, 1846], [142, 3937, 2174, 1080, 4034, 3667, 3875, 4290, 4612, 1174], [4321, 859, 2677, 1543, 4518, 4290, 4612, 1174], [4600, 1442, 4538, 3123, 2927, 2884, 209, 455, 4882, 3379], [1800, 2928, 2785], [57, 2010, 2287, 3753, 4723, 2285, 1791, 4382], [2453, 673, 3690, 4014, 2630, 3822, 3278, 3807, 

# Embedding Representation

In [22]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3799 2045 4992]
 [   0    0    0 ... 3997 3862 4382]
 [   0    0    0 ... 4439 3051 3532]
 ...
 [   0    0    0 ... 4290 4612 1174]
 [   0    0    0 ... 1191 4849 1849]
 [   0    0    0 ... 3937 4459 2371]]


In [23]:
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
embedded_docs

array([[   0,    0,    0, ..., 3799, 2045, 4992],
       [   0,    0,    0, ..., 3997, 3862, 4382],
       [   0,    0,    0, ..., 4439, 3051, 3532],
       ...,
       [   0,    0,    0, ..., 4290, 4612, 1174],
       [   0,    0,    0, ..., 1191, 4849, 1849],
       [   0,    0,    0, ..., 3937, 4459, 2371]], dtype=int32)

In [25]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [26]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

# Model Training

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.2, random_state=42)

In [28]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 11s 34ms/step - loss: 0.3071 - accuracy: 0.8604 - val_loss: 0.1974 - val_accuracy: 0.9169
Epoch 2/10
229/229 [==============================] - 9s 38ms/step - loss: 0.1346 - accuracy: 0.9481 - val_loss: 0.1944 - val_accuracy: 0.9185
Epoch 3/10
229/229 [==============================] - 9s 38ms/step - loss: 0.0988 - accuracy: 0.9623 - val_loss: 0.2269 - val_accuracy: 0.9174
Epoch 4/10
229/229 [==============================] - 7s 32ms/step - loss: 0.0759 - accuracy: 0.9720 - val_loss: 0.2403 - val_accuracy: 0.9136
Epoch 5/10
229/229 [==============================] - 10s 46ms/step - loss: 0.0538 - accuracy: 0.9822 - val_loss: 0.2662 - val_accuracy: 0.9144
Epoch 6/10
229/229 [==============================] - 8s 34ms/step - loss: 0.0379 - accuracy: 0.9873 - val_loss: 0.3149 - val_accuracy: 0.9095
Epoch 7/10
229/229 [==============================] - 10s 43ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.3560 - val_accuracy: 0.9

In [29]:
y_pred = model.predict(x_test)

115/115 [==============================] - 2s 7ms/step


In [30]:
y_pred

array([[9.9999374e-01],
       [2.4768754e-04],
       [3.7646639e-06],
       ...,
       [9.9999684e-01],
       [1.3522688e-05],
       [9.9999213e-01]], dtype=float32)

In [31]:
y_pred = np.where(y_pred > 0.6, 1, 0)

In [32]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9133169264424391
